In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from pathlib import Path
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
base_path = r"c:\Users\siddhi\Desktop\Projects-20240722T093004Z-001\Projects\ASL_detection\ASL_detection\extracted"
train_path = os.path.join(base_path, "asl_alphabet_train", "asl_alphabet_train")
test_path = os.path.join(base_path, "asl_alphabet_test", "asl_alphabet_test")

IMG_SIZE = 224  
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 29

class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
               'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
               'del', 'nothing', 'space']

print(f"Training path: {train_path}")
print(f"Test path: {test_path}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {class_names}")

In [ ]:
def count_images_per_class(data_path):
    """Count images in each class folder"""
    class_counts = {}
    for class_name in os.listdir(data_path):
        class_path = os.path.join(data_path, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            class_counts[class_name] = count
    return class_counts

train_counts = count_images_per_class(train_path)
print("Training images per class:")
for class_name, count in sorted(train_counts.items()):
    print(f"{class_name}: {count}")

total_train_images = sum(train_counts.values())
print(f"\nTotal training images: {total_train_images}")

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  # ASL gestures are orientation-specific
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Classes found: {len(train_generator.class_indices)}")
print(f"Class indices: {train_generator.class_indices}")

In [ ]:
def plot_sample_images(generator, num_samples=20):
    plt.figure(figsize=(20, 10))
    
    batch_images, batch_labels = next(generator)
    
    class_indices = generator.class_indices
    class_names_ordered = [k for k, v in sorted(class_indices.items(), key=lambda item: item[1])]
    
    for i in range(min(num_samples, len(batch_images))):
        plt.subplot(4, 5, i + 1)
        plt.imshow(batch_images[i])
        class_idx = np.argmax(batch_labels[i])
        plt.title(f"Class: {class_names_ordered[class_idx]}", fontsize=12)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

print("Sample training images:")
plot_sample_images(train_generator)

In [ ]:
#  transfer learning with EfficientNetB0
def create_asl_model():
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
  
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

model = create_asl_model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'top_3_accuracy']
)

print("Model created successfully!")
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_asl_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configured successfully!")

In [ ]:
print("Starting training...")
print(f"Training on {train_generator.samples} samples")
print(f"Validating on {validation_generator.samples} samples")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

print("\nTraining completed!")

In [ ]:
def plot_training_history(history):
    """Plot training and validation metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(history.history['loss'], label='Training Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    axes[1, 0].plot(history.history['top_3_accuracy'], label='Training Top-3 Accuracy')
    axes[1, 0].plot(history.history['val_top_3_accuracy'], label='Validation Top-3 Accuracy')
    axes[1, 0].set_title('Model Top-3 Accuracy')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Top-3 Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    if 'lr' in history.history:
        axes[1, 1].plot(history.history['lr'], label='Learning Rate')
        axes[1, 1].set_title('Learning Rate')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_yscale('log')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    else:
        axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
validation_generator.reset()
predictions = model.predict(validation_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = validation_generator.classes

class_indices = validation_generator.class_indices
class_names_ordered = [k for k, v in sorted(class_indices.items(), key=lambda item: item[1])]

print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_names_ordered))

accuracy = np.mean(predicted_classes == true_classes)
print(f"\nValidation Accuracy: {accuracy:.4f}")

top_3_predictions = np.argsort(predictions, axis=1)[:, -3:]
top_3_accuracy = np.mean([true_classes[i] in top_3_predictions[i] for i in range(len(true_classes))])
print(f"Top-3 Accuracy: {top_3_accuracy:.4f}")

In [ ]:
def plot_confusion_matrix(true_classes, predicted_classes, class_names, title='Confusion Matrix'):
    """Plot confusion matrix"""
    cm = confusion_matrix(true_classes, predicted_classes)
    
    plt.figure(figsize=(15, 12))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.title(title, fontsize=16)
    plt.xlabel('Predicted', fontsize=14)
    plt.ylabel('Actual', fontsize=14)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(true_classes, predicted_classes, class_names_ordered, 'ASL Classification Confusion Matrix')

In [ ]:
def test_on_official_testset():
    """Test the model on the official test images"""
    test_results = []
    
    test_files = [f for f in os.listdir(test_path) if f.lower().endswith('.jpg')]
    
    plt.figure(figsize=(20, 15))
    
    for i, test_file in enumerate(sorted(test_files)):
        img_path = os.path.join(test_path, test_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_normalized = img_resized / 255.0
        img_batch = np.expand_dims(img_normalized, axis=0)
        
        prediction = model.predict(img_batch, verbose=0)
        predicted_class_idx = np.argmax(prediction[0])
        confidence = prediction[0][predicted_class_idx]
        predicted_class = class_names_ordered[predicted_class_idx]
        
        true_class = test_file.replace('_test.jpg', '')
        
        test_results.append({
            'file': test_file,
            'true_class': true_class,
            'predicted_class': predicted_class,
            'confidence': confidence,
            'correct': true_class == predicted_class
        })
        
        if i < 28:
            plt.subplot(4, 7, i + 1)
            plt.imshow(img)
            color = 'green' if true_class == predicted_class else 'red'
            plt.title(f"True: {true_class}\nPred: {predicted_class}\nConf: {confidence:.3f}", 
                     color=color, fontsize=10)
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    test_accuracy = sum([r['correct'] for r in test_results]) / len(test_results)
    print(f"\nTest Set Results:")
    print(f"Test Accuracy: {test_accuracy:.4f} ({sum([r['correct'] for r in test_results])}/{len(test_results)})")
    
    incorrect = [r for r in test_results if not r['correct']]
    if incorrect:
        print(f"\nIncorrect Predictions:")
        for result in incorrect:
            print(f"  {result['file']}: True={result['true_class']}, Predicted={result['predicted_class']}, Confidence={result['confidence']:.3f}")
    else:
        print("\nAll test predictions are correct! 🎉")
    
    return test_results

test_results = test_on_official_testset()

In [ ]:
def predict_asl_sign(image_path, model, class_names, show_image=True):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img_normalized = img_resized / 255.0
    img_batch = np.expand_dims(img_normalized, axis=0)
    
    prediction = model.predict(img_batch, verbose=0)
    
    top_3_indices = np.argsort(prediction[0])[::-1][:3]
    top_3_classes = [class_names[i] for i in top_3_indices]
    top_3_confidences = [prediction[0][i] for i in top_3_indices]
    
    if show_image:
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Predicted: {top_3_classes[0]} (Confidence: {top_3_confidences[0]:.3f})")
        plt.axis('off')
        plt.show()
        
        print(f"Top 3 Predictions:")
        for i in range(3):
            print(f"  {i+1}. {top_3_classes[i]}: {top_3_confidences[i]:.3f}")
    
    return top_3_classes[0], top_3_confidences[0], list(zip(top_3_classes, top_3_confidences))

sample_test_image = os.path.join(test_path, "A_test.jpg")
if os.path.exists(sample_test_image):
    print("Testing on sample image:")
    predicted_class, confidence, top_3 = predict_asl_sign(sample_test_image, model, class_names_ordered)
else:
    print(f"Sample image not found at: {sample_test_image}")